In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json
import numpy as np

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD, createTrainSetAdapter_02_04_25_GOOD, createTestSetAdapter_02_04_25_GOOD
from src.utils.Corpus import CacheCorpus, FlatCorpus, Factory
from src.utils.DatasetManager import ProjectsDatasetManager
from src.Doc2Vec_model import Model, AnnoySearcher


from gensim.models.doc2vec import TaggedDocument
from annoy import AnnoyIndex

In [3]:
adapter = createAdapter_02_04_25_GOOD()#JSONMultiFileAdapter(CACHE_FILE_NAME)
manager = ProjectsDatasetManager(50, cacheAdapter = adapter)
cursor = CacheCorpus(manager, limit = 10)

flatAda = createTrainSetAdapter_02_04_25_GOOD()
flat = FlatCorpus(flatAda, limit = 10)

print(flat[[5]])

for doc in flat:
    print(doc)
    #flatAda.save([{"tokens" : doc.words, "tags" : doc.tags}])

#print(adapter.load(1))
#print(adapter.load(3))
#print(adapter.load(3))
#adapter.save([{"a" : [1,2,3,7,8], "b" : [4]}, {"a" : [9,5, 8], "b" : [6]}])

[TaggedDocument(words=['svelte', 'baidumaps', 'set', 'baidu', 'map', 'component', 'svelte'], tags=['github:vulcangz/svelte-baidumaps', 'svelte-baidumaps', 'Svelte', 'baidu-map', 'baidumap', 'map', 'svelte'])]
TaggedDocument<['python', 'djvulibre', 'python', 'interface', 'djvulibre'], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']>
TaggedDocument<['hledger', 'robust', 'fast', 'intuitive', 'plain', 'text', 'accounting', 'tool', 'cli', 'tui', 'web', 'interface'], ['github:simonmichael/hledger', 'hledger', 'Haskell', 'accounting', 'bounties', 'bounty', 'command-line-tool', 'console-application', 'finance', 'haskell', 'haskell-applications', 'hledger', 'ledger', 'plaintext', 'plaintext-accounting', 'plaintextaccounting', 'tui', 'web-application']>
TaggedDocument<['rule', 'nixpkgs', 'rule', 'importing', 'nixpkgs', 'package', 'bazel'], ['github:tweag/rules_nixpkgs', 'rules_nixpkgs', 'Starlark', 'bazel', 'bazel-rules', 'nix', 'nixpkgs']>
TaggedD

In [4]:
import logging
logging.basicConfig(
    #filename="/home/trukhinmaksim/src/logs/09-04-25_autotunning.log",
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO
)

In [7]:

trainAda = createTrainSetAdapter_02_04_25_GOOD()
train = Factory.createFlatTrainDBCorpus_02_04_25_GOOD(10)#FlatCorpus(trainAda, limit = 10)

testAda = createTestSetAdapter_02_04_25_GOOD()
test = Factory.createFlatTestDBCorpus_02_04_25_GOOD(4)#FlatCorpus(testAda, limit = 10)

VECTOR_SIZE = 40
EPOCHS_NUMBER = 2
WORD_MIN_COUNT = 1
WINDOW_SIZE = 7
NEGATIVE_SAMPLES_AMOUNT = 6
SUBSAMPLING_THRESHOLD = 1e-5
ALPHA_INIT = 0.05
ALPHA_FINAL = 0.00001
DM_DBOW_MODE = "DM" # "DBOW" "DM+DBOW"

# finetunning is done by twicking model parameters
model = Model(
    vector_size =  VECTOR_SIZE, 
    window =       WINDOW_SIZE, 
    min_count =    WORD_MIN_COUNT, 
    epochs =       EPOCHS_NUMBER, 
    dm_dbow_mode = DM_DBOW_MODE,
    negative =     NEGATIVE_SAMPLES_AMOUNT,
    sample =       SUBSAMPLING_THRESHOLD,
    alpha_init =   ALPHA_INIT,
    alpha_final =  ALPHA_FINAL
)

print(train[[0]][0])
print(test[[0]][0])
model.trainCorpus = train
model.testCorpus = test#[TaggedDocument(words = train[[1]][0].words, tags = train[[1]][0].tags)]

model.train()
#model.test()
#model.evaluate()

2025-04-17 08:59:30,975 : INFO : Delete adapter
2025-04-17 08:59:31,258 : INFO : Delete adapter
2025-04-17 08:59:31,320 : INFO : Model lifecycle event {'params': 'Model<dm/m,d40,n6,w7,s1e-05,t3>', 'datetime': '2025-04-17T08:59:31.320610', 'gensim': '4.3.3', 'python': '3.11.11 (main, Jan 14 2025, 05:22:51) [GCC 12.2.0]', 'platform': 'Linux-6.13.8-200.fc41.x86_64-x86_64-with-glibc2.36', 'event': 'created'}
2025-04-17 08:59:31,326 : INFO : collecting all words and their counts
2025-04-17 08:59:31,329 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2025-04-17 08:59:31,331 : INFO : collected 66 word types and 10 unique tags from a corpus of 10 examples and 86 words
2025-04-17 08:59:31,332 : INFO : Creating a fresh vocabulary
2025-04-17 08:59:31,333 : INFO : Model lifecycle event {'msg': 'effective_min_count=1 retains 66 unique words (100.00% of original 66, drops 0)', 'datetime': '2025-04-17T08:59:31.333356', 'gensim': '4.3.3', 'python': '3.11.11 (main,

TaggedDocument<['python', 'djvulibre', 'python', 'interface', 'djvulibre'], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre']>
TaggedDocument<['nagios', 'plugin', 'check', 'raid', 'nagios', 'icinga', 'sensu', 'plugin', 'check', 'current', 'server', 'raid', 'status'], ['github:glensc/nagios-plugin-check_raid', 'nagios-plugin-check_raid', 'Perl', 'icinga', 'icinga-plugin', 'monitoring', 'nagios', 'nagios-plugin', 'raid', 'sensu', 'sensu-plugin']>


In [6]:
2/0
#model.test(3)
s = AnnoySearcher.create(model.dv.vectors)

s.selectKmostSimilar(model.dv.vectors[1], 3)

ZeroDivisionError: division by zero

In [ ]:
model.selectKmostSimilar(v, 10)

In [ ]:
train.onlyID = False
model.dv.most_similar([v], topn = 10)

In [ ]:
topK = sorted(model.selectKmostSimilar(v, 10), key = lambda pair: pair[0])
topK

In [ ]:

trueRelevant = model.checkRelevants([p[0] for p in topK], ['github:FriedrichFroebel/python-djvulibre', 'python-djvulibre', 'Cython', 'djvu', 'djvulibre'])
trueRelevant

In [ ]:
train.onlyID = False
for doc in train:
    print(doc.tags)